<a href="https://colab.research.google.com/github/sarthak-chakraborty/TF-Lite/blob/master/ModelPersonalization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Note:
- After every execution of the tflite convert code and saving it in `custom_keras_model` directory, restart the kernel and execute the required cell.

- The whitelisted operators supported by tflite convert is given in [this link](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/lite/delegates/flex/whitelisted_flex_ops.cc)

- Link for the materials: [https://github.com/tensorflow/examples/tree/master/lite/examples/model_personalization](https://github.com/tensorflow/examples/tree/master/lite/examples/model_personalization)

In [1]:
# Clone the github repo
!git clone https://github.com/sarthak-chakraborty/examples.git #(Contains changes)

# Actual Repo
# !git clone https://github.com/tensorflow/examples.git

%cd examples/lite/examples/model_personalization/converter
!pip install -e .

Cloning into 'examples'...
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 10675 (delta 0), reused 0 (delta 0), pack-reused 10666
Receiving objects: 100% (10675/10675), 19.30 MiB | 32.30 MiB/s, done.
Resolving deltas: 100% (5475/5475), done.
/content/examples/lite/examples/model_personalization/converter
Obtaining file:///content/examples/lite/examples/model_personalization/converter
     |████████████████████████████████| 86.3MB 70kB/s 
     |████████████████████████████████| 2.1MB 45.5MB/s 
     |████████████████████████████████| 4.3MB 43.8MB/s 
     |████████████████████████████████| 501kB 39.8MB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: tensorflow 2.2.0
    Uninstalling tensorflow-2.2.0:
      Successfully uninstalled tensorflow-2.2.0
  Found existing installation: Pillow 7.0.0
 

1. Simple CNN Model Conversion to TFLite. All the operations have thir Ops defined in whitelisted_flex_ops.cc file.

In [2]:
# Imports
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.regularizers import l2
from tfltransfer import bases
from tfltransfer import heads
from tfltransfer import optimizers
from tfltransfer.tflite_transfer_converter import TFLiteTransferConverter

"""
Get the Base model
"""
base = tf.keras.Sequential([tf.keras.layers.InputLayer(input_shape=(224,224,3))])
base.save("base_model", save_format="tf")
base = bases.SavedModelBase("base_model")

"""
Simple Head Model with only Conv2D and MaxPool2D layers
"""
head = tf.keras.Sequential([
    layers.Conv2D(32, 3, input_shape=(224, 224, 3), padding='same'),
    layers.Activation('relu'),
    layers.MaxPool2D(),
    layers.Conv2D(32, 3, activation='relu', padding='same'),
    layers.MaxPool2D(),
    layers.Conv2D(64, 3, activation='relu', padding='same'),
    layers.MaxPool2D(),
    layers.Conv2D(64, 3, activation='relu', padding='same'),
    layers.GlobalAveragePooling2D(),
    layers.Dense(4, activation='softmax'),
])

# Optimizer is ignored by the converter! See docs.
head.compile(loss='categorical_crossentropy', optimizer='sgd')
converter = TFLiteTransferConverter(4,
                                    base,
                                    heads.KerasModelHead(head),
                                    optimizers.SGD(3e-2),
                                    train_batch_size=20)

converter.convert_and_save('custom_keras_model')

INFO:tensorflow:Assets written to: base_model/assets
Instructions for updating:
Please use `model.save(..., save_format="tf")` or `tf.keras.models.save_model(..., save_format="tf")`.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: None
INFO:tensorflow:Signatures INCLUDED in export for Train: ['train']
INFO:tensorflow:Signatures INCLUDED in export for Eval: None
INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow

/usr/local/lib/python3.6/dist-packages/tensorflow_core/lite/python/lite.py:854: UserWarning: Property target_ops is deprecated, please use target_spec.supported_ops instead.
  "target_spec.supported_ops instead." % name)


2. Next, we try to convert a shortened version of MobileNet to TFLite. Note that here, we use `DepthwiseConv2d` layer for which the Ops is not defined in whitelisted_flex_ops.cc.




In [1]:
# Imports
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.regularizers import l2
from tfltransfer import bases
from tfltransfer import heads
from tfltransfer import optimizers
from tfltransfer.tflite_transfer_converter import TFLiteTransferConverter

"""
Get the Base model
"""
base = tf.keras.Sequential([tf.keras.layers.InputLayer(input_shape=(224,224,3))])
base.save("base_model", save_format="tf")
base = bases.SavedModelBase("base_model")

"""
Mobile Net without BatchNorm
"""
model = tf.keras.Sequential()
model.add(layers.Conv2D(32, kernel_size=(3,3), strides=(1,1), input_shape=(224, 224, 3), padding='same', name='input_conv32'))
model.add(layers.Activation('relu', name='input_act'))

# (Channels, Strides)
int_layers = [
  (64, (1, 1)),
  (128, (2, 2))
  # (128, (1, 1)),
  # (256, (2, 2)),
  # (256, (1, 1)),
  # (512, (2, 2)),
  # *[(512, (1, 1)) for _ in range(5)],
  # (1024, (2, 2)),
  # (1024, (1, 1))
]
i=1
for channels, strides in int_layers:
  # Depthwise
  model.add(layers.DepthwiseConv2D(kernel_size=(3, 3), strides=strides, use_bias=False, padding='same', name='depth_conv{}_{}'.format(channels, i)))
  model.add(layers.Activation('relu', name='depth_act_{}'.format(i)))

  # Pointwise
  model.add(layers.Conv2D(channels, kernel_size=(1, 1), strides=(1, 1), use_bias=False, padding='valid', name='point_conv{}_{}'.format(channels, i)))
  model.add(layers.Activation('relu', name='point_act_{}'.format(i)))

  i += 1

model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dense(4, activation='softmax'))


head = model

# Optimizer is ignored by the converter! See docs.
head.compile(loss='categorical_crossentropy', optimizer='sgd')
converter = TFLiteTransferConverter(4,
                                    base,
                                    heads.KerasModelHead(head),
                                    optimizers.SGD(3e-2),
                                    train_batch_size=20)

converter.convert_and_save('custom_keras_model')

"""
On adding the line `converter.allow_custom_ops=True` at the desired location, it will get converted, since the converter expects that a custom operation is there

While training on android device the following error shows up:

Caused by: java.lang.IllegalArgumentException: Internal error: Failed to run on the given Interpreter: Encountered unresolved custom op: DepthwiseConv2dNativeBackpropInput.
  Node number 146 (DepthwiseConv2dNativeBackpropInput) failed to prepare.
"""

INFO:tensorflow:Assets written to: base_model/assets
Instructions for updating:
Please use `model.save(..., save_format="tf")` or `tf.keras.models.save_model(..., save_format="tf")`.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: None
INFO:tensorflow:Signatures INCLUDED in export for Train: ['train']
INFO:tensorflow:Signatures INCLUDED in export for Eval: None
INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow

/usr/local/lib/python3.6/dist-packages/tensorflow_core/lite/python/lite.py:854: UserWarning: Property target_ops is deprecated, please use target_spec.supported_ops instead.
  "target_spec.supported_ops instead." % name)


ConverterError: ignored

3a. One way to mitigate this is to replace DepthwiseConv2D layer into a combination of Conv2D layer.

In [1]:
# Imports
import tensorflow as tf
import numpy as np
from tensorflow.keras import layers
from tensorflow.keras.regularizers import l2
from tfltransfer import bases
from tfltransfer import heads
from tfltransfer import optimizers
from tfltransfer.tflite_transfer_converter import TFLiteTransferConverter

"""
Get the Base model
"""
base = tf.keras.Sequential([tf.keras.layers.InputLayer(input_shape=(32,32,3))])
base.save("base_model", save_format="tf")
base = bases.SavedModelBase("base_model")

"""
Mobile Net without BatchNorm
"""
x_in = tf.keras.Input(shape=(32,32,3))
x = layers.Conv2D(32, kernel_size=(3,3), strides=(1,1), padding='same', name='input_conv32')(x_in)
x = layers.Activation('relu', name='input_act')(x)

prev_channels = 32
# (Channels, Strides)
int_layers = [
  (64, (1, 1))
#   (128, (2, 2)),
#   (128, (1, 1)),
#   (256, (2, 2)),
#   (256, (1, 1)),
#   (512, (2, 2)),
#   *[(512, (1, 1)) for _ in range(5)],
#   (1024, (2, 2)),
#   (1024, (1, 1))
]
# int_layers=[(64, (1,1))]
i=1
for channels, strides in int_layers:
  # Depthwise
  x_depth = []
  print(i)
  for channel_num in range(prev_channels):
    sliced_tensor = x[:, :, :, channel_num]
    tensor = tf.expand_dims(sliced_tensor, axis=3)
    x_dummy = layers.Conv2D(1, kernel_size=(3,3), strides=strides, use_bias=False, padding='same', name='depth_conv{}-{}_{}'.format(channels, channel_num, i))(tensor)
    x_depth.append(x_dummy)
  x = tf.stack([x_depth[j][:, :, :, 0] for j in range(prev_channels)], axis=3)
  x = layers.Activation('relu', name='depth_act_{}'.format(i))(x)

  # Pointwise
  x = layers.Conv2D(channels, kernel_size=(1, 1), strides=(1, 1), use_bias=False, padding='valid', name='point_conv{}_{}'.format(channels, i))(x)
  x = layers.Activation('relu', name='point_act_{}'.format(i))(x)

  prev_channels = channels
  i += 1

x = layers.GlobalAveragePooling2D()(x)
x_out = layers.Dense(10, activation='softmax')(x)
model = tf.keras.Model(inputs=x_in, outputs=x_out)


head = model

# Optimizer is ignored by the converter! See docs.
head.compile(loss='categorical_crossentropy', optimizer='sgd')
converter = TFLiteTransferConverter(10,
                                    base,
                                    heads.KerasModelHead(head),
                                    optimizers.SGD(3e-2),
                                    train_batch_size=4)

converter.convert_and_save('custom_keras_model')

INFO:tensorflow:Assets written to: base_model/assets
1
Instructions for updating:
Please use `model.save(..., save_format="tf")` or `tf.keras.models.save_model(..., save_format="tf")`.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: None
INFO:tensorflow:Signatures INCLUDED in export for Train: ['train']
INFO:tensorflow:Signatures INCLUDED in export for Eval: None
INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorfl

/usr/local/lib/python3.6/dist-packages/tensorflow_core/lite/python/lite.py:854: UserWarning: Property target_ops is deprecated, please use target_spec.supported_ops instead.
  "target_spec.supported_ops instead." % name)


3b. The other way is to freeze the particular layer. Here we change the model description while keeping a DepthwiseConv2D in the model definition.

In [1]:
# Imports
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import Conv2D, DepthwiseConv2D, MaxPool2D, BatchNormalization, Flatten, Dropout, Dense, Activation, ReLU, Add, Concatenate, InputSpec
from tensorflow.keras import Input, Model

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.utils import to_categorical

from tfltransfer import bases
from tfltransfer import heads
from tfltransfer import optimizers
from tfltransfer.tflite_transfer_converter import TFLiteTransferConverter

"""
Get the Base model
"""
base = tf.keras.Sequential([tf.keras.layers.InputLayer(input_shape=(224,224,3))])
base.save("base_model", save_format="tf")
base = bases.SavedModelBase("base_model")


"""
Head Model
"""
x_in = Input(shape=(224,224,3), name='input')
x = Conv2D(32, kernel_size=(3, 3), padding='valid', use_bias=False, name='conv32_1')(x_in)
x = Activation('relu')(x)
x = Conv2D(32, kernel_size=(3,3), padding='same', use_bias=False, name='conv32_2')(x)
x = Activation('relu')(x)

x = Conv2D(64, kernel_size=(3, 3), padding='valid', use_bias=False, name='conv64_1')(x)
x = Activation('relu')(x)
x = DepthwiseConv2D(kernel_size=(3,3), strides=(2,2), padding='same', use_bias=False, name='depthwise_conv_1')(x)
x = Conv2D(64, kernel_size=(1, 1), padding='valid', use_bias=False, name='pointwise_conv64_1')(x)
x = Activation('relu')(x)
x = Dropout(0.25)(x)

x = Flatten()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.25)(x)
x_out = Dense(4, activation='softmax')(x)

model = Model(inputs=x_in, outputs=x_out, name='my_model')


# Here 7th layer is the DepthwiseConv2D layer
model.layers[7].trainable = False


for variable in model.trainable_variables:
    print(variable.name)

head = model

# Optimizer is ignored by the converter! See docs.
head.compile(loss='categorical_crossentropy', optimizer='sgd')
converter = TFLiteTransferConverter(4,
                                    base,
                                    heads.KerasModelHead(head),
                                    optimizers.SGD(3e-2),
                                    train_batch_size=20)

converter.convert_and_save('custom_keras_model')

INFO:tensorflow:Assets written to: base_model/assets
conv32_1/kernel:0
conv32_2/kernel:0
conv64_1/kernel:0
pointwise_conv64_1/kernel:0
dense/kernel:0
dense/bias:0
dense_1/kernel:0
dense_1/bias:0
Instructions for updating:
Please use `model.save(..., save_format="tf")` or `tf.keras.models.save_model(..., save_format="tf")`.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: None
INFO:tensorflow:Signatures INCLUDED in export for Train: ['train']
INFO:tensorflow:Signatures INCLUDED in export for Eval: None
INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
IN

/usr/local/lib/python3.6/dist-packages/tensorflow_core/lite/python/lite.py:854: UserWarning: Property target_ops is deprecated, please use target_spec.supported_ops instead.
  "target_spec.supported_ops instead." % name)


ConverterError: ignored

3c. Here, we will freeze all the layers that are below the DepthwiseConv2D layer, that is, all the layers from Input to DepthwiseConv2D.


In [1]:
# Imports
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import Conv2D, DepthwiseConv2D, MaxPool2D, BatchNormalization, Flatten, Dropout, Dense, Activation, ReLU, Add, Concatenate, InputSpec
from tensorflow.keras import Input, Model

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.utils import to_categorical

from tfltransfer import bases
from tfltransfer import heads
from tfltransfer import optimizers
from tfltransfer.tflite_transfer_converter import TFLiteTransferConverter

"""
Get the Base model
"""
base = tf.keras.Sequential([tf.keras.layers.InputLayer(input_shape=(224,224,3))])
base.save("base_model", save_format="tf")
base = bases.SavedModelBase("base_model")


"""
Head Model
"""
x_in = Input(shape=(224,224,3), name='input')
x = Conv2D(32, kernel_size=(3, 3), padding='valid', use_bias=False, name='conv32_1')(x_in)
x = Activation('relu')(x)
x = Conv2D(32, kernel_size=(3,3), padding='same', use_bias=False, name='conv32_2')(x)
x = Activation('relu')(x)

x = Conv2D(64, kernel_size=(3, 3), padding='valid', use_bias=False, name='conv64_1')(x)
x = Activation('relu')(x)
x = DepthwiseConv2D(kernel_size=(3,3), strides=(2,2), padding='same', use_bias=False, name='depthwise_conv_1')(x)
x = Conv2D(64, kernel_size=(1, 1), padding='valid', use_bias=False, name='pointwise_conv64_1')(x)
x = Activation('relu')(x)
x = Dropout(0.25)(x)

x = Flatten()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.25)(x)
x_out = Dense(4, activation='softmax')(x)

model = Model(inputs=x_in, outputs=x_out, name='my_model')


# Here 7th layer is the DepthwiseConv2D layer
for i in range(len(model.layers)):
    if i <= 7:
        model.layers[i].trainable=False

for variable in model.trainable_variables:
    print(variable.name)

head = model

# Optimizer is ignored by the converter! See docs.
head.compile(loss='categorical_crossentropy', optimizer='sgd')
converter = TFLiteTransferConverter(4,
                                    base,
                                    heads.KerasModelHead(head),
                                    optimizers.SGD(3e-2),
                                    train_batch_size=20)

converter.convert_and_save('custom_keras_model')

INFO:tensorflow:Assets written to: base_model/assets
pointwise_conv64_1/kernel:0
dense/kernel:0
dense/bias:0
dense_1/kernel:0
dense_1/bias:0
Instructions for updating:
Please use `model.save(..., save_format="tf")` or `tf.keras.models.save_model(..., save_format="tf")`.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: None
INFO:tensorflow:Signatures INCLUDED in export for Train: ['train']
INFO:tensorflow:Signatures INCLUDED in export for Eval: None
INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:Signatures INCLUDED in export for Classi

/usr/local/lib/python3.6/dist-packages/tensorflow_core/lite/python/lite.py:854: UserWarning: Property target_ops is deprecated, please use target_spec.supported_ops instead.
  "target_spec.supported_ops instead." % name)


4. Here, we try to convert a BatchNorm Layer to TFLite. However, as we see, it will give an error.

In [1]:
# Imports
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.regularizers import l2
from tfltransfer import bases
from tfltransfer import heads
from tfltransfer import optimizers
from tfltransfer.tflite_transfer_converter import TFLiteTransferConverter

"""
Get the Base model
"""
base = tf.keras.Sequential([tf.keras.layers.InputLayer(input_shape=(224,224,3))])
base.save("base_model", save_format="tf")
base = bases.SavedModelBase("base_model")

"""
BatchNorm Layer
"""

model = tf.keras.Sequential()
model.add(layers.Conv2D(32, kernel_size=(3,3), strides=(1,1), input_shape=(224, 224, 3), padding='same'))
model.add(layers.BatchNormalization())
model.add(layers.Activation('relu'))

model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dense(4, activation='softmax'))

head = model


# Optimizer is ignored by the converter! See docs.
head.compile(loss='categorical_crossentropy', optimizer='sgd')
converter = TFLiteTransferConverter(4,
                                    base,
                                    heads.KerasModelHead(head),
                                    optimizers.SGD(3e-2),
                                    train_batch_size=20)

converter.convert_and_save('custom_keras_model')


INFO:tensorflow:Assets written to: base_model/assets
Instructions for updating:
Please use `model.save(..., save_format="tf")` or `tf.keras.models.save_model(..., save_format="tf")`.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: None
INFO:tensorflow:Signatures INCLUDED in export for Train: ['train']
INFO:tensorflow:Signatures INCLUDED in export for Eval: None
INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow

AttributeError: ignored

In [ ]:
# Imports
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.regularizers import l2
from tfltransfer import bases
from tfltransfer import heads
from tfltransfer import optimizers
from tfltransfer.tflite_transfer_converter import TFLiteTransferConverter


# # Functional API
# x_in = tf.keras.layers.Input(shape=(224,224,3))
# x = layers.Conv2D(32, kernel_size=(3,3), strides=(1,1), padding='valid')(x_in)
# x = layers.Activation('relu')(x_in)
# x = layers.GlobalAveragePooling2D()(x)
# x_out = layers.Dense(4, activation='softmax')(x)
# model = tf.keras.Model(inputs=x_in, outputs=x_out)


# Sequential API
model = tf.keras.Sequential()
model.add(layers.Conv2D(32, kernel_size=(3,3), strides=(1,1), input_shape=(224,224,3), padding='valid'))
model.add(layers.Activation('relu'))
model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dense(4, activation='softmax'))


import tensorflow.compat.v1 as tfv1
import os
import shutil

out_model_dir = 'tmp'
shutil.rmtree(out_model_dir)
if not os.path.isdir(out_model_dir):
  os.makedirs(out_model_dir)

tf.keras.experimental.export_saved_model(model, out_model_dir)

with tfv1.Session(graph=tf.Graph()) as sess:
  metagraph = tfv1.saved_model.load(sess, tags=[tf.saved_model.SERVING], export_dir=out_model_dir)
  _eval_signature = metagraph.signature_def.get('serving_default')

print(_eval_signature)

INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: ['serving_default']
INFO:tensorflow:Signatures INCLUDED in export for Train: None
INFO:tensorflow:Signatures INCLUDED in export for Eval: None
INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: tmp/saved_model.pb
INFO:tensorflow:Restoring parameters from tmp/variables/variables
inputs {
  key: "conv2d_12_input"
  value {
    name: "conv2d_12_input:0"
    dtype: DT_FLOAT
    tensor_shape {
      dim {
        size: -1
      }
      dim {
        size: 224
      }
      dim {
        size: 224
      }
      dim {
        size: 3
      }
    }
  }
}
outputs {
  key: "dense_12"
  value {
    name: "dense_12/Softmax:0"
    dtype: DT_FLOAT
    tensor_shape {
      dim {
        size: -1
      }
      dim {
        size: 4
      }
    }
  }
}
method_